In [20]:
import pandas as pd

In [132]:
# Downloading the csv file from your account
url = 'https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/falls1.csv'
 
snow = pd.read_csv(url)
snow.isna().sum()

Unnamed: 0      0
date            0
snow_depth    176
month           0
year            0
dtype: int64

In [133]:
snow.fillna(method='ffill', inplace=True)
snow.isna().sum()

Unnamed: 0    0
date          0
snow_depth    1
month         0
year          0
dtype: int64

In [134]:
snow.head()

,Unnamed: 0,date,snow_depth,month,year
0,1,2020-06-05,NaN,6,2020
1,2,2020-06-06,0.0,6,2020
2,3,2020-06-07,0.0,6,2020
3,4,2020-06-08,0.0,6,2020
4,5,2020-06-09,0.0,6,2020


In [128]:
def get_git_csv_temp(url):
    '''Reads in a csv file which is assumed to have the following columns
    'Product code'
    'Bureau of Meteorology station number'
    'Year'
    'Month'
    'Day'
    'Maximum temperature (Degree C)'
    'Days of accumulation of maximum temperature'
    'Quality'
    
       This function does two things: 
       1) Drops Unwanted Columns 
       2) Parses Date Column

       Source texts have a special label, -1.
       :param csv_file: The directory for the file_information.csv file
       :return: A dataframe with date and Maximum temperature'''
    
    # Pass the csv file to a create a dataframe for pre processing.
    df = pd.read_csv(url)
    
    df = df.drop(columns = ['Product code',
                            'Bureau of Meteorology station number',
                            'Days of accumulation of maximum temperature',
                            'Quality'])
    # Parse the date column
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    
    # Drop the unwanted columns
    df = df.drop(columns = ['Year', 'Month', 'Day'])
    
    return df

In [129]:
def get_git_csv_rain(url):
    '''Reads in a csv file which is assumed to have the following columns
    'Product code'
    'Bureau of Meteorology station number'
    'Year'
    'Month'
    'Day'
    'Maximum temperature (Degree C)'
    'Days of accumulation of maximum temperature'
    'Quality'
    
       This function does two things: 
       1) Drops Unwanted Columns 
       2) Parses Date Column

       Source texts have a special label, -1.
       :param csv_file: The directory for the file_information.csv file
       :return: A dataframe with date and Maximum temperature'''
    
    # Pass the csv file to a create a dataframe for pre processing.
    df = pd.read_csv(url)
    
    df = df.drop(columns = ['Product code',
                            'Bureau of Meteorology station number',
                            'Period over which rainfall was measured (days)',
                            'Quality'])
    # Parse the date column
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    
    # Drop the unwanted columns
    df = df.drop(columns = ['Year', 'Month', 'Day'])
    
    return df

In [130]:
temp_2016 = get_git_csv_temp('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/MAXT_IDCJAC0010_083084_2016_Data.csv')
temp_2017 = get_git_csv_temp('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/MAXT_IDCJAC0010_083084_2017_Data.csv')
temp_2018 = get_git_csv_temp('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/MAXT_IDCJAC0010_083084_2018_Data.csv')
temp_2019 = get_git_csv_temp('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/MAXT_IDCJAC0010_083084_2019_Data.csv')
temp_2020 = get_git_csv_temp('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/MAXT_IDCJAC0010_083084_2020_Data.csv')
temp_2021 = get_git_csv_temp('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/MAXT_IDCJAC0010_083084_2021_Data.csv')

rain_2016 = get_git_csv_rain('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/RAIN_IDCJAC0009_083084_2016_Data.csv')
rain_2017 = get_git_csv_rain('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/RAIN_IDCJAC0009_083084_2017_Data.csv')
rain_2018 = get_git_csv_rain('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/RAIN_IDCJAC0009_083084_2018_Data.csv')
rain_2019 = get_git_csv_rain('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/RAIN_IDCJAC0009_083084_2019_Data.csv')
rain_2020 = get_git_csv_rain('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/RAIN_IDCJAC0009_083084_2020_Data.csv')
rain_2021 = get_git_csv_rain('https://raw.githubusercontent.com/boffyd/UdacityML-Capstone/main/Data/RAIN_IDCJAC0009_083084_2021_Data.csv')


temp = pd.concat([temp_2016,temp_2017,temp_2018,temp_2019,temp_2020,temp_2021])
rain = pd.concat([rain_2016,rain_2017,rain_2018,rain_2019,rain_2020,rain_2021])

weather = temp.merge(rain, on='Date', how='left')

In [131]:
weather.head()

,Maximum temperature (Degree C),Date,Rainfall amount (millimetres)
0,21.2,2016-01-01,0.0
1,15.4,2016-01-02,1.8
2,9.7,2016-01-03,9.4
3,8.5,2016-01-04,14.0
4,12.4,2016-01-05,17.8


In [135]:
weather.dtypes

Maximum temperature (Degree C)           float64
Date                              datetime64[ns]
Rainfall amount (millimetres)            float64
dtype: object

In [136]:
snow.dtypes

Unnamed: 0      int64
date           object
snow_depth    float64
month           int64
year            int64
dtype: object